In [ ]:
import sqlite3
import tweepy as tw
from datetime import datetime
import pandas as pd

## Create a class that extract data from tweet object and store it to database

In [ ]:
'''by: wishnu002@gmail.com'''

class tweetDB:
    '''
    Interface class to connect this python program
    to specified sqlite db file
    '''
    
    def __init__(self, dbName):
        self.dbName = dbName   
        
    def openConnection(self):
        self.connection = sqlite3.connect(self.dbName)
        self.cursor = self.connection.cursor()
        
    def closeConnection(self):
        self.cursor.close()
        self.connection.close()
        
    def commitConnection(self):
        self.connection.commit()
        
    def save_user(self, tweetObject):        
               
        self.u_id_ = tweetObject.user.id
        self.u_name = tweetObject.user.name
        self.u_sName = tweetObject.user.screen_name
        
        if tweetObject.user.location == '':
            self.u_loc = 'Unspecified'
        else:
            self.u_loc = tweetObject.user.location
        
        self.acc_createdDate = tweetObject.user.created_at.strftime('%d-%m-%Y')
        self.u_follower = tweetObject.user.followers_count
        self.u_friend = tweetObject.user.friends_count        
        self.u_verified = int(tweetObject.user.verified) #return 1 if true, 0 if false
                
        query = '''INSERT INTO
        User(userid, name, screenname, location, accountcreated, follower, friend, verified)
        VALUES (?,?,?,?,?,?,?,?);'''
        
        input_list = (self.u_id_, self.u_name, self.u_sName, self.u_loc,
                      self.acc_createdDate, self.u_follower, self.u_friend, self.u_verified)

        
        '''
        The column 'userid' of the table 'User' in tweetDB has 'unique',
        if duplicate is inputted, it will raise an error.
        
        Since we don't want any duplicates in our 'User' table, we can utilize this
        error to ignore the insert operation.
        '''
        try:
            self.cursor.execute(query, input_list)
        except:
            pass
        

    def save_tweet(self, tweetObject):
        '''
        Storing tweet data from tweet object to database,
        one tweet object at a time
        '''
        
        self.t_id = tweetObject.id
        self.u_id =  tweetObject.user.id
        self.t_date = tweetObject.created_at.strftime('%d-%m-%Y')
        self.t_text = tweetObject.full_text
        self.t_retweet = tweetObject.retweet_count
        self.t_fave = tweetObject.favorite_count
        
        query = '''INSERT INTO Tweet(tweetid, userid, createddate, tweet, retweeted, favorited)
        VALUES (?,?,?,?,?,?);'''
        
        input_list = (self.t_id, self.u_id, self.t_date, self.t_text, self.t_retweet, self.t_fave)
                
        try:
            self.cursor.execute(query, input_list)
        except:
            pass

        
    def safeExecution_save(self, tweetObject):
        '''
        Ensures userid in User table exist before making entry to
        Tweet table (because there is userid FK column in tweet table)
        '''
        
        self.save_user(tweetObject)
        self.save_tweet(tweetObject)

## Twitter mining section

In [ ]:
keys = pd.read_excel('twitterAPI_keys2.xlsx')

api_key = keys['API_key'][0]
api_secretKey = keys['API_secret_key'][0]
accessToken = keys['access_token'][0]
accessToken_secret = keys['access_token_secret'][0]

auth = tw.OAuthHandler(api_key, api_secretKey)
auth.set_access_token(accessToken, accessToken_secret)

twitter = tw.API(auth)

In [ ]:
'''
Specifying the search query and number of
maximum tweet data retrieved per search operation.
'''

query = '#gamestonk #gme -discord -doge -BTC -coin AND -filter:retweets'

max_tweets = 4000 #max tweet per search result

In [ ]:
'''
Tweet search "until" parameter are made per day to anticipate if number of search result
of a certain day is more than the max tweet number that specified above.
'''

import time

#search tweets posted before 29 Jan 2021.
search_result1 = [status for status in tw.Cursor(twitter.search, q=query, count=100, until='2021-1-29', tweet_mode='extended').items(max_tweets)]
#hold operation for 20 minutes (+5 minutes for safe margin) due to twitter API request limitation.
time.sleep(1200)

#search tweets posted before 30 Jan 2021
search_result2 = [status for status in tw.Cursor(twitter.search, q=query, count=100, until='2021-1-30', tweet_mode='extended').items(max_tweets)]
time.sleep(1200)

#search tweets posted before 31 Jan 2021
search_result3 = [status for status in tw.Cursor(twitter.search, q=query, count=100, until='2021-1-31', tweet_mode='extended').items(max_tweets)]
time.sleep(1200)

#search tweets posted before 1 Feb 2021
search_result4 = [status for status in tw.Cursor(twitter.search, q=query, count=100, until='2021-2-1', tweet_mode='extended').items(max_tweets)]
time.sleep(1200)

#search tweets posted before 2 Feb 2021
search_result5 = [status for status in tw.Cursor(twitter.search, q=query, count=100, until='2021-2-2', tweet_mode='extended').items(max_tweets)]

print('Search Result 1 (until 28-Jan-21 23:59)', len(search_result1))
print('Search Result 2 (until 29-Jan-21 23:59)', len(search_result2))
print('Search Result 3 (until 30-Jan-21 23:59)', len(search_result3))
print('Search Result 4 (until 31-Jan-21 23:59)', len(search_result4))
print('Search Result 5 (until 01-Jan-21 23:59)', len(search_result5))

'''
Start processing on 2 Feb 2021,
No more data mining
'''

Search Result 1 (until 28-Jan-21 23:59) 1377
Search Result 2 (until 29-Jan-21 23:59) 2092
Search Result 3 (until 30-Jan-21 23:59) 2365
Search Result 4 (until 31-Jan-21 23:59) 2584
Search Result 5 (until 01-Jan-21 23:59) 3006


In [ ]:
'''
Create search result backup
'''

import joblib

joblib.dump(search_result1, 'search_result1.sav')
joblib.dump(search_result2, 'search_result2.sav')
joblib.dump(search_result3, 'search_result3.sav')
joblib.dump(search_result4, 'search_result4.sav')
joblib.dump(search_result5, 'search_result5.sav')

## Tweet data extraction and storing execution

In [ ]:
'''
Create an instance of tweetDB object
'''

db = tweetDB('tweet_db_gamestonk_final2.db')

In [ ]:
'''
Write all the result into the database file
'''

db.openConnection()

for item in search_result1:
    db.safeExecution_save(item)
    
for item in search_result2:
    db.safeExecution_save(item)
    
for item in search_result3:
    db.safeExecution_save(item)
    
for item in search_result4:
    db.safeExecution_save(item)
    
for item in search_result5:
    db.safeExecution_save(item)
    
db.commitConnection()
db.closeConnection()

'''
There won't be any duplicate user and tweet entry stored in database even when
the tweet data is overlapping between search results, thank's to the 'UNIQUE' property
of the column 'userid' and 'tweetid' in the database.
'''